# L7: Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [4]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [5]:
import os
api_key = os.getenv("API_KEY")
os.environ["OPENAI_API_KEY"] = api_key


os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## crewAI Tools

In [6]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
# read_resume = FileReadTool(file_path='./fake_resume.md')
# semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')
read_resume = FileReadTool(file_path='./GirishMohite-Resume.md')
semantic_search_resume = MDXSearchTool(mdx='./GirishMohite-Resume.md')

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [ ]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [7]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)

In [8]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [9]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)

## Creating Tasks

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


## Creating the Crew

In [15]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
job_application_inputs = {
    # 'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'job_posting_url':'https://www.jobserve.com/gb/en/search-jobs-in-London,-London,-United-Kingdom/AI-SOLUTIONS-ENGINEER-8422AA4400AECC9745/',
    'github_url': 'https://github.com/gmohite7',
    'personal_writeup': """
    Working as a Engineering manager and Data Architect for Multi cloud management platform, Girish working with the SME’s from the IT management designed data models for IT Health management. He also spearheaded in building integrations to a wide variety of data sources to ingest structured/semi structured/unstructured data. Girish shaped a strategic vision for the next gen service desk driven by enhanced analytics, automation and cognitive capabilities by working closely with Architects, UX designers, Developers, Offering managers and sponsored users. Designed and delivered various aspects on data ingestion leveraging ELK platform. This constituted but not limited in building integrations with wide variety of sources such as AWS, Azure, GCP, IBM Cloud,DataDog,Dynatrace,Prometheus,SNOW etc. Also played a pivot role in bringing about Observability and Orchestration in the data ingestion layer. In addition to this built many solutions that helped address the Data quality and the Data catalogue on the data lake."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://www.jobserve.com/gb/en/search-jobs-in-London,-London,-United-Kingdom/AI-SOLUTIONS-ENGINEER-8422AA4400AECC9745/) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/gmohite7) URLs, and personal write-up (
    Working as a Engineering manager and Data Architect for Multi cloud management platform, Girish working with the SME’s from the IT management designed data models for IT Health management. He also spearheaded in building integrations to a wide variety of data sources to ingest structured/semi structured/unstructured data. Girish shaped a strategic vi

- Dislplay the generated `tailored_resume.md` file.

In [18]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

Girish B Mohite
Cell +91 7406466678 • girish_mohite@hotmail.com

Solution Architect & Engineering Manager

Professional Experience

Kyndryl Solutions, India - Solution Architect & Engineering Manager (Sep 2021 – current)
IBM GTS Lab, India – Chief Programmer & Engineering Manager, GTS Labs (Dec 2016 -Aug 2021)
IBM Hursley Lab, UK – Technical Lead, Technology, Operation Tools Transformation (Aug 2008 – Nov 2016)
Patterson, Salisbury, UK – Senior Software Engineer, Logon2 payroll Application (Aug 2007 – July 2008)
IBM Hursley Lab, UK – Senior Consultant (Jan 2005 – July 2008)
IBM Greenock Lab, UK – Software Developer, Edinburgh congestion project (Mar 00 – Dec 2004)
Experience: 20+ years; Product, Data Architect and Engineering Manager: 6+ years; Business Leadership: 8 + years;

Key Achievements as Solution Architect & Engineering Manager

Led Data Integration, Data Quality, Data Pipe extension for over 190+ data sources
Onboarded and provided Data operation support for over 1500+ clients across 6 GEO’s
Global Operation of 6 data Lake regions with regulatory compliance like GDPR, LGPD, CCPA
Awarded with IBM Corporate award for outstanding contributions to technology & business impact
Managed a portfolio with an overall budget ~ $80 M+ and new business growth to a tune of ~250 + FTEs
Strong interpersonal, team, and client relationship building and presentation skills
Successful setup of new delivery unit, executed large and complex Data Center and cloud migration project
Employee Recognition Award in IBM & consistent Top Performer Recognition through career history
Proven skill in communicating with and engaging multiple stakeholders, C-level execs, Enterprise architects, and SMEs
Experience in upskill and business growth
Led a delivery team of Product Manager, Project Managers, Engineering, and other functional teams to deliver end to end of release phases
Experience working on Scrum/Agile environments tools like Jira, RTC, Github
SPOC from offering and Engagement team for Delivery Review, releases from content to GA
Trusted advisor to report on the state of the release to executive stakeholders in checkpoint reviews
Interlock with engineering team to identify and remove cross-product roadblocks early in the project cycle
Worked on multiple projects concurrently in a fast-paced environment and drove changes across engineering groups
Exceptional skills influencing, teaming, and delivering solutions in a highly complex and nebulous environment
Identifies, assesses, and manages risks to the success of the program to achieve strategic objectives
Leadership and responsibility for the delivery of the program - including scope, schedule, budget, quality, and business value
Defines and executes program governance to ensure appropriate controls and visibility
Worked extensively with senior Executives on product strategy, organization initiatives, program reviews, and quality management
Perform End to End program management starting from inception to customer site deployment of software products
Experience in interacting with VP's, CTO, and higher Planning and Budgeting discussions
Demonstrated experience managing cross-functional teams to deliver comprehensive offerings or programs to market
Collaborative, but confident leader who can ensure support from a diverse set of internal contributors to the overall mission of building a robust, external developer community
Technical expertise sufficient to enable thoughtful solutions to problems in the developer community context
Managed multiple high-value, high-complexity programs e.g. Programs which have high Risk in form of contract, multi-geo & client implementation

CAREER SNAPSHOT

Apr’20 – Present: Engineering Manager / Solution Architect – MCMP – GTS Labs

Working as an Engineering manager and Data Architect for Multi-cloud management platform
Designed data models for IT Health management
Spearheaded in building integrations to a wide variety of data sources
Shaped a strategic vision for the next-gen service desk driven by enhanced analytics, automation, and cognitive capabilities
Designed and delivered various aspects on data ingestion leveraging ELK platform
Built integrations with a wide variety of sources such as AWS, Azure, GCP, IBM Cloud, DataDog, Dynatrace, Prometheus, SNOW, etc.
Played a pivotal role in bringing about Observability and Orchestration in the data ingestion layer
Built solutions to address Data quality and the Data catalogue on the data lake

Dec 16 – Mar’20: Chief Programmer Cognitive Delivery Insights – GTS Labs

Led development engineering and delivery across a wide range of stakeholders
Developed Cognitive Insights with a focus on AI, ML, and Data Operations using big data technologies

Cognitive Delivery Insights (CDI): 190+ Data sources, 800+ ETL 60+ Insights, 50+ Data Models
CDI is the GTS data lake and insights tool for IT Service Delivery
Operating in 6 geos, supporting 1000+ customers
CDI collects data that includes IT Assets, Service Management, Event Management both in real-time and batch modes
Produces business intelligence – cognitive Delivery ‘Insights’
Curates, classifies, and links data sets to provide business intelligence – cognitive ‘Insights’
Accelerates IT operations with AI and agile

IBM GTS: IBM Services IT Console
Provides end-to-end visibility across monitoring systems, service management systems, third party service providers
Incorporates multiple reporting engines and integrates several vendor monitoring systems
Provides unprecedented visibility to both clients and service delivery professionals

Sep’ 15- Dec’16 – Technical Lead IBM Hursley Lab, UK

Managed the design and development of operational tools for acquisition integration and transformation projects
Targeted towards OSS clearance projects for acquisitions, source code, and defect management system migrations
Responsible for detailed analysis, modelling business processes, educating teams, key decisions, and managing delivery of automated reports

Aug’08 to Sep’15, Technical Lead IBM Hursley Lab, UK

Modeled the business process for managing Intellectual property for an acquired portfolio of products
Implemented a complete end-to-end workflow
Responsible for detailed analysis, modelling business processes, educating teams, and automating workflow stages

Aug 07 to Jul’08 – Software Developer Patterson Ltd, Salisbury UK

Involved in the design and development of the Lenovo GL Project integrated into Logon2 application
Designed and developed a banking application for payment files processing
Developed a framework for generating payment files in International ACH format
Designed and Developed Struts Action classes, Web services, and more

Apr’06 to Jul’08, Software Developer IBM Hursley Lab, UK

Designed and developed the Planning Portal for advisory services
Integrated Quick Address 5.2 for post code look-up
Improved the planning portal search tool and facilitated automatic transfer of application & User Information
Provided support, maintenance, and problem fixes

Jan’05 to Mar’06 Technical Lead IBM Hursley Lab, UK

Developed Etrace, a wireless and web-based solution
Implemented application-based security and database migration
Coding, unit testing, and quality assurance using Hibernate, JSP’s, and Webservices

SIGNATURE SKILLS & CORE COMPETENCIES

Data Architecture, Data modeling, Design
Managing multiple teams, leaders, architects, TPM’s
IT Service Delivery, ITIL & Operations
OO Analysis & Design, OO Design Patterns
Transition & Transformation Planning and Execution
IT Service Design and Release management
Mentoring & Coaching, Java, C++, Python, SQL
IT Technology – Virtualization, cloud – Private, Public and Hybrid
Delivery Management on ITIL framework
Bigdata Stack - Hadoop, Elastic Search, Logstash, Kibana
IT Engineering and Offering Product development
Datacenter Transition, Transformation & Consolidation
DevOps – Jira, RTC, Github, Kubernetes

EDUCATION, TRAININGS & CERTIFICATIONS

Bachelors in Electronics Engineering
Google Professional Cloud Architect
Sun/Oracle certified Enterprise Architect
Certified (PMP) Project Management Professional

Place: Bangalore (Girish Mohite)

- Dislplay the generated `interview_materials.md` file.

In [19]:
display(Markdown("./interview_materials.md"))

Based on the resume of Girish Mohite and the job requirements, here are some potential interview questions and talking points to help the candidate prepare for the initial interview:

1. Can you discuss your experience with RAG (Retrieval-Augmented Generation) and how you have implemented it in your previous roles?
2. How have you utilized LLM reasoning in your work, and what benefits have you seen from incorporating it into your projects?
3. Can you provide examples of your proficiency in Python and JavaScript, and how you have leveraged these skills to solve complex problems?
4. How have you applied prompt engineering in your previous roles, and what strategies have you used to optimize prompt generation?
5. Can you share your experience working in a consulting or professional services environment, and how it has shaped your approach to problem-solving and client interactions?
6. How has your technology or business-related degree contributed to your success in your career, and what specific skills have you gained from your educational background?
7. Can you elaborate on your experience as a Director of AIOps Engineering and how you have utilized your skills in DevOps, Cloud Architecture, and Technical Project Management in this role?
8. How have you demonstrated your ability to work effectively with cross-functional teams and different disciplines, and what strategies have you used to foster collaboration and achieve successful project outcomes?
9. Can you provide examples of your leadership experience in managing high-value, high-complexity programs, and how you have navigated challenges and risks to achieve strategic objectives?
10. How have you contributed to the growth and upskilling of your teams, and what methods have you used to mentor and coach team members to drive professional development?

These questions and talking points are designed to help the candidate showcase their skills, experience, and achievements in alignment with the job requirements and demonstrate their ability to excel in the role they are applying for.

# CONGRATULATIONS!!!

## Share your accomplishment!
- Once you finish watching all the videos, you will see the "In progress" image on the bottom left turn into "Accomplished".
- Click on "Accomplished" to view the course completion page with your name on it.
- Take a screenshot and share on LinkedIn, X (Twitter), or Facebook.  
- **Tag @Joāo (Joe) Moura, @crewAI, and @DeepLearning.AI, (and a few of your friends if you'd like them to try out the course)**
- **Joāo and DeepLearning.AI will "like"/reshare/comment on your post!**

## Get a completion badge that you can add to your LinkedIn profile!
- Go to [learn.crewai.com](https://learn.crewai.com).
- Upload your screenshot of your course completion page.
- You'll get a badge from CrewAI that you can share!

(Joāo will also talk about this in the last video of the course.)